# 🧑‍🏫️ Multi-label NER Training (Scientific Method)

In [ ]:
#@title Define Hyperparameters & Configuration

BATCH_SIZE = 16 #@param
LEARNING_RATE = 5e-5 #@param
MAX_EPOCHS = 20 #@param
THRESHOLD = 0.5 #@param
PATIENCE = 3 #@param {type:"integer"}

# For focal loss function
ALPHA = 0.65 #@param
GAMMA = 3.5 #@param

In [ ]:
#@title Install Dependencies

!pip install transformers pytorch-lightning torchmetrics scikit-learn seaborn matplotlib


### 💾 Pre-Processing and Data Loading

We will load the dataset, merge the multi-label annotations, and perform a determinstic train/test split.

In [ ]:
#@title Load & Transform Data
import json
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# Path configuration - adjust if needed
BASE_DIR = Path('..')
DATASET_PATH = BASE_DIR / 'dataset' / 'new_dataset.json'
TEST_SET_OUTPUT_PATH = BASE_DIR / 'dataset' / 'held_out_test_set.json'

def load_and_transform_data(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # The JSON has a 'bias_dataset' key
    if 'bias_dataset' in data:
        data = data['bias_dataset']
    
    transformed_data = []
    for entry in data:
        text_str = entry['text_str']
        # Determine sequence length (approximate based on raw split, tokenization fixes later)
        # The annotations are usually character or token aligned. Let's inspect the first one.
        # 'annotations': {'GEN': [...], 'STEREO': [...], 'UNFAIR': [...]}
        
        annotations = entry['annotations']
        # Verify generic length using GEN (assuming all lists are same length)
        seq_len = len(annotations['GEN'])
        
        # Merge tags into list of lists
        combined_tags = []
        for i in range(seq_len):
            tags_for_token = []
            # Check each category
            if annotations['GEN'][i] != 'O':
                tags_for_token.append(annotations['GEN'][i])
            if annotations['STEREO'][i] != 'O':
                tags_for_token.append(annotations['STEREO'][i])
            if annotations['UNFAIR'][i] != 'O':
                tags_for_token.append(annotations['UNFAIR'][i])
            
            if not tags_for_token:
                tags_for_token.append('O')
            
            combined_tags.append(tags_for_token)
        
        transformed_data.append({
            'text_str': text_str,
            'ner_tags': combined_tags,
            'id': entry['id']
        })
    
    return transformed_data

# 1. Load Data
if not DATASET_PATH.exists():
    raise FileNotFoundError(f"Dataset not found at {DATASET_PATH}")

all_data = load_and_transform_data(DATASET_PATH)
print(f"Loaded {len(all_data)} samples.")
print("Sample Entry 0:", all_data[0])

# 2. Deterministic Split (Train / Test)
# Since dataset is small (200), we hold out 20% (40 samples) for pure testing.
train_val_data, test_data = train_test_split(all_data, test_size=0.20, random_state=42, shuffle=True)

print(f"Train/Val Size: {len(train_val_data)}")
print(f"Test Size: {len(test_data)}")

# Save Test Set for Reproducibility
with open(TEST_SET_OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump(test_data, f, indent=2)
print(f"Saved held-out test set to {TEST_SET_OUTPUT_PATH}")

ner_annotations_df = pd.DataFrame(train_val_data)


In [ ]:
#@title Tokenization Logic
import torch
from transformers import BertTokenizerFast

# init tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

label2id = {
    'O': 0,
    'B-STEREO': 1, 'I-STEREO': 2,
    'B-GEN': 3, 'I-GEN': 4,
    'B-UNFAIR': 5, 'I-UNFAIR': 6
}
num_labels = len(label2id)
id2label = {v: k for k, v in label2id.items()}

def tokenize_and_align_labels(text, annotations, tokenizer, label2id, max_length=128):
    # The text is already tokenized in the source? Or is it a raw string?
    # Based on the JSON logic, 'annotations' are aligned to *original* tokens (likely whitespace split).
    # We must assume `text.split()` aligns with `annotations` length.
    
    original_words = text.split()
    if len(original_words) != len(annotations):
        # Simple heuristic fix or skip
        # print(f"Warning: Length mismatch. Text: {len(original_words)}, Labels: {len(annotations)}")
        # Truncate to shorter
        min_len = min(len(original_words), len(annotations))
        original_words = original_words[:min_len]
        annotations = annotations[:min_len]

    tokenized_inputs = tokenizer(original_words, is_split_into_words=True, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
    word_ids = tokenized_inputs.word_ids()

    aligned_labels = []
    for word_idx in word_ids:
        if word_idx is None:
            aligned_labels.append([-100] * num_labels)
        else:
            current_tags = annotations[word_idx]
            label_vector = [0] * num_labels
            for tag in current_tags:
                if tag in label2id:
                    label_vector[label2id[tag]] = 1
            aligned_labels.append(label_vector)
            
    return tokenized_inputs, aligned_labels

def preprocess_data(df, tokenizer, label2id, max_length=128):
    tokenized_inputs_list = []
    aligned_labels_list = []
    for _, row in df.iterrows():
        text = row['text_str']
        annotations = row['ner_tags']
        inputs, labels = tokenize_and_align_labels(text, annotations, tokenizer, label2id, max_length)
        tokenized_inputs_list.append(inputs)
        aligned_labels_list.append(labels)
    return tokenized_inputs_list, aligned_labels_list

# Preprocess Train/Val Data
tokenized_texts, labels = preprocess_data(ner_annotations_df, tokenizer, label2id)
print(f"Processed {len(tokenized_texts)} training samples.")

In [ ]:
#@title Lightning Data Module
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl

class NERDataset(Dataset):
    def __init__(self, tokenized_texts, labels):
        self.input_ids = [t['input_ids'].squeeze() for t in tokenized_texts]
        self.attention_mask = [t['attention_mask'].squeeze() for t in tokenized_texts]
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx], torch.tensor(self.labels[idx])

class NERDataModule(pl.LightningDataModule):
    def __init__(self, tokenized_texts, labels, batch_size=16, val_split=0.15):
        super().__init__()
        self.dataset = NERDataset(tokenized_texts, labels)
        self.batch_size = batch_size
        self.val_split = val_split

    def setup(self, stage=None):
        val_size = int(len(self.dataset) * self.val_split)
        train_size = len(self.dataset) - val_size
        self.train_ds, self.val_ds = random_split(self.dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=True, num_workers=0)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, num_workers=0)

data_module = NERDataModule(tokenized_texts, labels, batch_size=BATCH_SIZE)


### 🧠 Model Definition with Early Stopping

In [ ]:
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import BertForTokenClassification, get_linear_schedule_with_warmup
from pytorch_lightning import LightningModule
from torchmetrics.classification import MultilabelPrecision, MultilabelRecall, MultilabelF1Score
from torchmetrics import HammingDistance

class NERModel(LightningModule):
    def __init__(self, learning_rate=5e-5, threshold=0.5, alpha=0.75, gamma=3):
        super().__init__()
        self.save_hyperparameters()
        self.bert = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=7)
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.alpha = alpha 
        self.gamma = gamma
        
        # Metrics (ignoring 'O' class usually, but keeping simple here)
        self.f1 = MultilabelF1Score(num_labels=4, average='macro', threshold=0.5)
        self.hamming = HammingDistance(task='multilabel', num_labels=4, threshold=0.5)

    def forward(self, input_ids, attention_mask):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask).logits

    def focal_loss(self, logits, labels):
        # Flatten: (batch * seq_len, num_labels)
        logits = logits.view(-1, 7)
        labels = labels.view(-1, 7).float()
        
        # Mask ignored tokens (-100)
        mask = (labels[:, 0] != -100) # Assuming if first dim is -100 whole vec is ignored. Or verify mask logic.
        # Actually, in our vector encoding, -100 is likely passed as [-100, -100...]. 
        # Let's check a sum condition or similar. 
        # Safe check: valid vectors have 0 or 1. Invalid have -100.
        valid_mask = (labels >= 0).all(dim=1)
        
        logits = logits[valid_mask]
        labels = labels[valid_mask]
        
        if logits.shape[0] == 0:
             return torch.tensor(0.0, device=logits.device, requires_grad=True)

        bce_loss = F.binary_cross_entropy_with_logits(logits, labels, reduction='none')
        p_t = torch.exp(-bce_loss)
        loss = self.alpha * (1 - p_t) ** self.gamma * bce_loss
        return loss.mean()

    def training_step(self, batch, batch_idx):
        outputs = self(batch[0], batch[1])
        loss = self.focal_loss(outputs, batch[2])
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(batch[0], batch[1])
        loss = self.focal_loss(outputs, batch[2])
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=int(0.1 * self.trainer.estimated_stepping_batches),
            num_training_steps=self.trainer.estimated_stepping_batches
        )
        return [optimizer], [scheduler]


In [ ]:
#@title Train with Early Stopping
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import shutil

# Clean logs
shutil.rmtree('lightning_logs', ignore_errors=True)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints',
    filename='gusnet-best-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
    save_last=True
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    verbose=True,
    mode='min'
)

model = NERModel()
trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    callbacks=[checkpoint_callback, early_stop_callback],
    accelerator='auto',
    devices=1,
    enable_progress_bar=True
)

trainer.fit(model, data_module)


In [ ]:
#@title Evaluate on Held-Out Test Set

# Load best model
best_model_path = checkpoint_callback.best_model_path
print(f"Loading best model from {best_model_path}")
best_model = NERModel.load_from_checkpoint(best_model_path)
best_model.eval()
best_model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Prepare Test Data
test_inputs = []
test_labels = []
test_df = pd.DataFrame(test_data)

test_in, test_lbl = preprocess_data(test_df, tokenizer, label2id)
test_ds = NERDataset(test_in, test_lbl)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)

# Inference Loop
from sklearn.metrics import classification_report, accuracy_score

all_preds = []
all_trues = []

device = best_model.device
with torch.no_grad():
    for batch in test_dl:
        input_ids = batch[0].to(device)
        mask = batch[1].to(device)
        labels = batch[2].cpu().numpy()
        
        logits = best_model(input_ids, mask)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs > THRESHOLD).astype(int)
        
        # Flatten and filter padded -100
        for i in range(len(labels)):
            valid_indices = np.where((labels[i] >= 0).all(axis=1))[0]
            if len(valid_indices) > 0:
                p = preds[i][valid_indices]
                l = labels[i][valid_indices]
                all_preds.extend(p)
                all_trues.extend(l)

all_preds = np.array(all_preds)
all_trues = np.array(all_trues)

print("\n--- Test Set Results ---")
# Calculate per-class metrics
target_names = ['O', 'B-STEREO', 'I-STEREO', 'B-GEN', 'I-GEN', 'B-UNFAIR', 'I-UNFAIR']
print(classification_report(all_trues, all_preds, target_names=target_names, zero_division=0))

# Entity Level Custom Metrics (Hamming etc)
print(f"Exact Match Pct: {accuracy_score(all_trues, all_preds):.4f}")
